<a href="https://colab.research.google.com/github/marcoss00/fundamentos_ia_data_science/blob/master/Atividade_3_Sample_distribution_and_sampling_distribution%2C_TLC_and_p_value.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A: sample distribution and sampling distribution
A partir do que foi feito na Atividade 2 “Considerando o primeiro trabalho, defina um experimento em que dados são coletados de uma variável de interesse do seu trabalho”, crie  uma população artificial dessa variável com uma distribuição Gaussiana e demonstre sample distribution and sampling distribution. Faça avaliações de probabilidade de possíveis eventos.

Baixando o pacote que permite buscar datasets diretamente do repositório UCI

In [1]:
pip install ucimlrepo

Importação das bibliotecas para cálculos numéricos, gráficos, estatística e manipulação de dados.

In [2]:
from ucimlrepo import fetch_ucirepo
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import norm

In [3]:
# Busca o dataset
cardiotocography = fetch_ucirepo(id=193)

# Guarda as features e as classes em dataframe
df_features = cardiotocography.data.features
df_class = cardiotocography.data.targets

# Junta as features e as classes em um unico dataframe
df = pd.concat([df_features, df_class], axis=1)

In [5]:
# Filtra a feature 'LB' onde a classe 'NSP' for 1 (normal)
acc_per_sec_normal = df[df['NSP'] == 1]['LB']

# Calcula a média e o desvio padrão dos dados reais filtrados
mean_real = np.mean(acc_per_sec_normal)
std_real = np.std(acc_per_sec_normal)

# Define a seed de geração de numeros aleatorios
# Serve para fixar a aleatoriedade, garantindo que o resultado dos números aleatórios seja sempre o mesmo toda vez que o código for executado.
np.random.seed(42)
# Quantidade de amostras
n_samples = 100
# Geração dos dados artificiais com distribuição normal com base nos dados reais
dados_artificiais = np.random.normal(loc=mean_real, scale=std_real, size=n_samples)

# Ajusta os dados simulados para obter média e desvio padrão estimados
mu, sigma = norm.fit(dados_artificiais)